# Пример использования Binance US Data Framework

Этот ноутбук демонстрирует использование фреймворка для загрузки и хранения исторических данных Binance US с возможностью работы как с локальной SQLite базой данных, так и с облачной базой данных Google Cloud SQL.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Пример использования Binance US Data Framework в Google Colab.
Скопируйте этот код в ячейку Colab и запустите.
"""

# Установка фреймворка и зависимостей
!pip install python-binance pandas ipywidgets matplotlib duckdb sqlalchemy pymysql cryptography

# Клонирование репозитория и установка фреймворка
!git clone git+https://github.com/St0rmMaster/binance_data_framework.git
%cd binance_data_framework

!pip install -e .
%cd ..

# Импорт необходимых библиотек
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from IPython.display import display, HTML
from binance_data_framework import BinanceUSClient, LocalDataManager, DataDownloaderUI

# Инициализация клиентов
# Можно использовать ключи API, если они доступны
# from google.colab import userdata
# api_key = userdata.get('BINANCE_US_API_KEY')
# api_secret = userdata.get('BINANCE_US_API_SECRET')
# api_client = BinanceUSClient(api_key=api_key, api_secret=api_secret)

api_client = BinanceUSClient(api_key=None, api_secret=None)
connection_status = api_client.connect()

if connection_status:
    print("Успешное подключение к Binance US API")
else:
    print("Ошибка подключения к Binance US API")
    raise Exception("Не удалось подключиться к API")

# Инициализация менеджера базы данных
db_manager = LocalDataManager(db_path='binance_data.db')

# Создание и отображение интерактивного интерфейса
ui = DataDownloaderUI(api_client, db_manager)
ui.display()

# Примечание: После запуска интерфейса, вы можете выбрать торговую пару, 
# таймфрейм, период и загрузить данные. 
# При желании, вы можете добавить дополнительный код для 
# программного получения данных ниже. 

## Интеграция с Google Cloud SQL

Для использования Google Cloud SQL необходимо выполнить несколько шагов:

1. Создать проект в Google Cloud Platform
2. Активировать Cloud SQL API
3. Создать экземпляр MySQL в Cloud SQL
4. Настроить доступ и создать пользователя базы данных
5. Установить необходимые зависимости

После этого можно использовать CloudDataManager для хранения данных в Google Cloud SQL.

In [ ]:
# Установка дополнительных зависимостей для работы с Google Cloud SQL
!pip install sqlalchemy pymysql cryptography google-cloud-storage google-cloud-sql

# Импорт необходимых классов для работы с облачной БД
from binance_data_framework import CloudDataManager, DataSynchronizer

### Аутентификация в Google Cloud

Для работы с Google Cloud SQL необходимо выполнить аутентификацию. В Google Colab есть несколько способов аутентификации:

1. Через сервисный аккаунт (рекомендуется)
2. Через интерактивную аутентификацию пользователя

Ниже приведен пример интерактивной аутентификации.

In [ ]:
# Аутентификация в Google Cloud Platform
from google.colab import auth
auth.authenticate_user()

### Использование Cloud SQL Proxy (опционально)

Для более безопасного соединения с базой данных можно использовать Cloud SQL Proxy. Это особенно полезно при работе вне среды Google Cloud.

In [ ]:
# Установка Cloud SQL Proxy (опционально)
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy

# Запуск Cloud SQL Proxy для подключения к Cloud SQL
# !./cloud_sql_proxy -instances=YOUR_CONNECTION_NAME=tcp:3306 &

# Пример:
# !./cloud_sql_proxy -instances=your-project:us-central1:binance-data=tcp:3306 &

### Подключение к Google Cloud SQL

Теперь можно создать экземпляр CloudDataManager для работы с базой данных Google Cloud SQL.

In [ ]:
# Параметры подключения к Cloud SQL
# Замените эти значения на свои
project_id = 'your-project-id'  # ID вашего проекта в Google Cloud
region = 'us-central1'          # Регион, где размещен экземпляр
instance_name = 'binance-data'  # Имя экземпляра Cloud SQL
db_name = 'binance_db'          # Имя базы данных
db_user = 'root'                # Имя пользователя
db_password = 'your-password'   # Пароль пользователя

# Если используете Cloud SQL Proxy, установите use_proxy=True
use_proxy = True  # Установите False для прямого подключения в Cloud Run или AppEngine

# Создаем экземпляр CloudDataManager
try:
    cloud_manager = CloudDataManager(
        db_user=db_user,
        db_password=db_password,
        db_name=db_name,
        project_id=project_id,
        instance_name=instance_name,
        region=region,
        use_proxy=use_proxy
    )
    print("Успешное подключение к Google Cloud SQL")
except Exception as e:
    print(f"Ошибка подключения к Google Cloud SQL: {e}")

## Синхронизация данных между локальной и облачной базой

С помощью класса DataSynchronizer можно синхронизировать данные между локальной SQLite базой и облачной базой Google Cloud SQL.

In [ ]:
# Создаем экземпляр синхронизатора данных
try:
    sync_manager = DataSynchronizer(
        local_manager=db_manager,
        cloud_manager=cloud_manager
    )
    
    # Синхронизация данных из локальной базы в облачную
    print("Синхронизация из локальной базы в облачную...")
    sync_stats = sync_manager.sync_local_to_cloud()
    print(f"Синхронизировано {sync_stats['symbols_synced']} символов, {sync_stats['records_synced']} записей")
    print(f"Ошибок: {sync_stats['errors']}")
    
    # Синхронизация данных из облачной базы в локальную
    # print("Синхронизация из облачной базы в локальную...")
    # sync_stats = sync_manager.sync_cloud_to_local()
    # print(f"Синхронизировано {sync_stats['symbols_synced']} символов, {sync_stats['records_synced']} записей")
    # print(f"Ошибок: {sync_stats['errors']}")
except Exception as e:
    print(f"Ошибка при синхронизации данных: {e}")

## Пример использования данных из облачного хранилища

Вы можете использовать CloudDataManager так же, как и LocalDataManager, для чтения и записи данных.

In [ ]:
# Пример получения данных из облачного хранилища
symbol = 'BTCUSDT'
timeframe = '1h'
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

try:
    # Проверяем наличие данных в облачной базе
    data_exists, date_range = cloud_manager.check_data_exists(symbol, timeframe, start_date, end_date)
    
    if data_exists:
        # Получаем данные из облачной базы
        print(f"Получение данных для {symbol} на таймфрейме {timeframe} из облачного хранилища")
        df = cloud_manager.get_data(symbol, timeframe, start_date, end_date)
        
        if not df.empty:
            print(f"Получено {len(df)} свечей")
            print("\nПервые 5 строк:")
            display(df.head())
            
            # Визуализация данных
            plt.figure(figsize=(12, 6))
            plt.plot(df.index, df['close'])
            plt.title(f'{symbol} - {timeframe} (Cloud Storage)')
            plt.grid(True)
            plt.show()
    else:
        # Данных нет, загружаем из API и сохраняем в облачную базу
        print(f"Данные для {symbol} не найдены в облачной базе, загрузка из API...")
        df = api_client.get_historical_data(symbol, timeframe, start_date, end_date)
        
        if not df.empty:
            # Сохраняем данные в облачную базу
            cloud_manager.save_data(df, symbol, timeframe)
            print(f"Данные сохранены в облачную базу")
            
            # Визуализация данных
            plt.figure(figsize=(12, 6))
            plt.plot(df.index, df['close'])
            plt.title(f'{symbol} - {timeframe} (Cloud Storage)')
            plt.grid(True)
            plt.show()
except Exception as e:
    print(f"Ошибка при работе с облачным хранилищем: {e}")

## Информация о данных в облачном хранилище

Вы можете получить информацию о всех сохраненных данных в Google Cloud SQL.

In [ ]:
# Получение информации о сохраненных данных в облачной базе
try:
    cloud_info = cloud_manager.get_stored_info()
    
    if not cloud_info.empty:
        print("Информация о данных в облачном хранилище:")
        display(cloud_info)
        
        # Получаем статистику по символам и таймфреймам
        symbols = cloud_info['symbol'].unique()
        timeframes = cloud_info['timeframe'].unique()
        
        print(f"Всего символов: {len(symbols)}")
        print(f"Всего таймфреймов: {len(timeframes)}")
        
        # Вычисляем общий размер данных
        total_days = 0
        for _, row in cloud_info.iterrows():
            days = (row['end_date'] - row['start_date']).total_seconds() / (60 * 60 * 24)
            total_days += days
            
        avg_size_per_day_mb = 0.01  # Примерный размер данных за один день в МБ
        estimated_size_mb = total_days * avg_size_per_day_mb * len(timeframes)
        
        print(f"Приблизительный размер данных: {estimated_size_mb:.2f} МБ")
    else:
        print("В облачной базе нет сохраненных данных")
except Exception as e:
    print(f"Ошибка при получении информации о данных в облачном хранилище: {e}")

## Закрытие соединений

Важно закрывать соединения с базами данных после завершения работы.

In [ ]:
# Закрытие соединений с базами данных
try:
    db_manager.close()
    print("Соединение с локальной базой данных закрыто")
except:
    pass

try:
    cloud_manager.close()
    print("Соединение с облачной базой данных закрыто")
except:
    pass